# First, Evolve 2 populations as normal

### Setup

In [542]:
from deap import base, creator, tools
import random
import string
import itertools

signals = list(string.ascii_lowercase)  # Characters used to make words
meanings = list(range(0, 100))  # 100 possible meanings

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", dict, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

def create_word(min_length=1, max_length=3):
    length = random.randint(min_length, max_length)
    return ''.join(random.choice(signals) for _ in range(length))

def create_individual():
    individual = creator.Individual()
    individual.is_bilingual = 0
    individual.myLang = 0
    individual.local_state = random.choice(meanings)
    individual.age = 0
    for meaning in meanings:

        word = create_word()
        individual[meaning] = word

    return individual

toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


### Evaluation

In [543]:
GLOBAL_STATE = random.choice(meanings)

"""
Adds newly heard word to individual's vocabulary 
if a randomly guessed meaning matches that of the 
speaker.
"""
def learn(individual, new_word, speakers_meaning):
    random_meaning = random.choice(meanings)
    if random_meaning == speakers_meaning:
        individual[random_meaning] = new_word

def pairwise_communication(speaker, listener, GLOBAL_STATE):
    fitness = 0
    # speaker encodes signals
    local_meaning = speaker.local_state
    global_meaning = GLOBAL_STATE

    semantic_local = speaker[local_meaning]
    semantic_global = speaker[global_meaning]

    # only update if listener has words in dictionary
    decoded_local = -100
    decoded_global = -101

    localWordPresentinListener = 0
    globalWordPresentinListener = 0
    # listener decodes local signal or possibly learns new word
    for meaning in meanings:
        if semantic_local == listener[meaning]:  
            decoded_local = meaning
            localWordPresentinListener = 1
        if semantic_global == listener[meaning]:
            decoded_global = meaning
            globalWordPresentinListener = 1

    if localWordPresentinListener == 0:
        if (random.random() < 1):
            learn(listener, semantic_local, local_meaning)

    if globalWordPresentinListener == 0:
        if (random.random() < 1): 
            learn(listener, semantic_global, global_meaning)

    if (decoded_local == local_meaning) & (localWordPresentinListener == 1):
        fitness += 1
    if (decoded_global == global_meaning) & (globalWordPresentinListener == 1):
        fitness += 1

    # penalty if different ideas are represented by the same word
    if local_meaning != global_meaning:
        if semantic_global == semantic_local:
            fitness -= 1
        if decoded_local == decoded_global:
            fitness -= 1
        
    return fitness   
            
def survival_task(individual, GROUP_SIZE):
    penalty = 0
    for i in range(2 * (GROUP_SIZE - 1)): 
        meaning1, meaning2 = random.sample(meanings, 2) # two different meanings
        if individual[meaning1] == individual[meaning2]: # same word
            penalty += 2

    return penalty  

# evaluate group
def evaluate_group(group, GROUP_SIZE):
    GLOBAL_STATE = random.choice(meanings)
    # reset fitness
    for ind in group:
        # ind.fitness.values = (0,)  # reset fitness
        ind.local_state = random.choice(meanings)

    # speak
    for speaker, listener in itertools.permutations(group, 2):
        fitness_bonus = pairwise_communication(speaker, listener, GLOBAL_STATE)
        speaker.fitness.values = (speaker.fitness.values[0] + fitness_bonus,)
        listener.fitness.values = (listener.fitness.values[0] + fitness_bonus,)

    # individual survival
    for ind in group:
        penalty = survival_task(ind, GROUP_SIZE)
        ind.fitness.values = (ind.fitness.values[0] - penalty,)


### Declare population + initial eval

In [544]:
# create pop
# baseline: N=100, G=10
population = toolbox.population(n=100)

for ind in population:
    ind.fitness.values = (0,)

toolbox.register("evaluate", evaluate_group)

GROUP_SIZE = 10

# for i in range(500):
groups = [population[i:i + GROUP_SIZE] for i in range(0, len(population), GROUP_SIZE)]
for group in groups:
    toolbox.evaluate(group, GROUP_SIZE)

count = 0
for ind in population:
    # print(ind)
    if ind.fitness.values[0] > 0:
        print(ind, ind.fitness.values[0])
        count += 1
print(count)

{0: 'flb', 1: 't', 2: 'v', 3: 't', 4: 'tm', 5: 'jmp', 6: 'l', 7: 'rb', 8: 'm', 9: 'y', 10: 'au', 11: 'fls', 12: 'jzh', 13: 'hof', 14: 'f', 15: 'yc', 16: 'gpj', 17: 'w', 18: 'f', 19: 'yxb', 20: 'vh', 21: 'cz', 22: 'boi', 23: 'vqf', 24: 'j', 25: 'ogs', 26: 'm', 27: 'e', 28: 'v', 29: 'e', 30: 'rk', 31: 'ox', 32: 'jhn', 33: 'q', 34: 'dqw', 35: 'az', 36: 'dwf', 37: 'j', 38: 'osl', 39: 's', 40: 'pgf', 41: 'ah', 42: 'b', 43: 'qlu', 44: 'wk', 45: 'huv', 46: 'hh', 47: 'rgx', 48: 'h', 49: 'ywb', 50: 'ken', 51: 'zaw', 52: 'lj', 53: 'y', 54: 'gvj', 55: 'o', 56: 'j', 57: 'g', 58: 'c', 59: 'tpg', 60: 'g', 61: 'hmg', 62: 'lg', 63: 'af', 64: 'otm', 65: 'xrm', 66: 'vpi', 67: 'y', 68: 'cqy', 69: 'ul', 70: 'l', 71: 'pl', 72: 'bo', 73: 'o', 74: 'wqv', 75: 'gxr', 76: 'p', 77: 'nq', 78: 'z', 79: 'he', 80: 'e', 81: 'mi', 82: 'eo', 83: 'y', 84: 'yg', 85: 'or', 86: 'bsf', 87: 'ik', 88: 'hd', 89: 'c', 90: 'o', 91: 'g', 92: 'j', 93: 'i', 94: 'mto', 95: 'hlt', 96: 'hy', 97: 'aia', 98: 'wla', 99: 'w'} 1.0
{0: 'mpb

### Params

In [232]:
from deap import algorithms
import numpy

# Parameters
NUM_GENERATIONS = 15000
CXPB = 0.5  # Crossover probability (.5)
MUTPB = 0.01  # .01 best (100 meanings))

# mut for signals
def mutateInd(individual, indpb):
    for i in range(len(meanings)):
        if random.random() < indpb:
            individual[i] = create_word()

    return individual,

# tools
toolbox.register("mate", tools.cxUniform, indpb=0.5)
toolbox.register("mutate", mutateInd, indpb=0.1)
toolbox.register("select", tools.selRoulette) # inspired by paper
toolbox.register("tourn", tools.selTournament)
toolbox.register("randsel", tools.selRandom)
toolbox.register("stochastic_select", tools.selStochasticUniversalSampling)



stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

# for steady state
def select_next_generation(parents, offspring, max_age=3):
    combined = parents + offspring
    filtered = [ind for ind in combined if ind.age < max_age]
    selected = toolbox.tourn(filtered, len(population), 2)
    return selected

def calculate_average_agreement(population, num_meanings):
    total_agreement = 0
    for i in range(num_meanings):
        gene_values = [individual[i] for individual in population]
        most_common_value, count = Counter(gene_values).most_common(1)[0]
        agreement_percentage = (count / len(population)) * 100
        total_agreement += agreement_percentage
    average_agreement = total_agreement / num_meanings
    return average_agreement




### run algorithm

In [211]:
for gen in range(NUM_GENERATIONS):
    
    #for ES:
    MU = 100
    LAMBDA = 150

    """
    Mating selection
    """
    #offspring = toolbox.randsel(population, LAMBDA)
    #offspring = toolbox.select(population, len(population))
    #offspring = toolbox.tourn(population, len(population), tournsize=2)
    offspring = toolbox.randsel(population, len(population)) # random sel

    offspring = list(map(toolbox.clone, offspring))

    # mutation and crossover
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    """
    Evaluate Fitness
    """
    random.shuffle(offspring)
    for ind in offspring:
        ind.fitness.values = (0,)
    groups = [offspring[i:i + GROUP_SIZE] for i in range(0, len(offspring), GROUP_SIZE)]
    for group in groups:
        toolbox.evaluate(group, GROUP_SIZE)

    for ind in population:
        ind.age += 1

    """
    Survivor Selection
    """
    # ES:
    #population[:] = toolbox.stochastic_select(offspring, MU)

    # steady state:
    population [:] = select_next_generation(population, offspring) 

    # generational:
    #population[:] = offspring


    fits = [ind.fitness.values[0] for ind in population]


    from collections import Counter

    def count_duplicate_values(individual):
        value_count = Counter(individual.values())
        duplicates = [value for value, count in value_count.items() if count > 1]
        return len(duplicates)  # Return the number of unique values that are duplicates

    dup_list = []

    for individual in population:
        num_duplicates = count_duplicate_values(individual)
        dup_list.append(num_duplicates)
       
    sum_dups = 0
    for dup in dup_list:
        sum_dups += dup
    #print("Dup Avg:", sum_dups / len(ind) / len(population))



    length = len(population)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5

    print(f"Generation {gen}: Min {min(fits)}, Max {max(fits)}, Avg {mean}, Std {std}, Avg % of duplicate genes across participants {sum_dups / len(ind) / len(population)}")
    agreement = calculate_average_agreement(population, 100)
    print(calculate_average_agreement(population, 100))
    if (mean == 36.0) & (max(fits) == 36.0) & (min(fits) == 36.0) & (std == 0.0) & (agreement == 100.0) & ((sum_dups / len(ind) / len(population)) == 0.0):
        break
# print final pop
for ind in population:
    print(ind, ind.fitness.values)

Generation 0: Min -2.0, Max 3.0, Avg 0.41, Std 0.9065870063044142, Avg % of duplicate genes across participants 0.1002
6.72
Generation 1: Min 0.0, Max 5.0, Avg 0.88, Std 1.1855800268223147, Avg % of duplicate genes across participants 0.10039999999999999
10.65
Generation 2: Min 0.0, Max 6.0, Avg 1.66, Std 1.5376605607220342, Avg % of duplicate genes across participants 0.1008
21.03
Generation 3: Min -2.0, Max 18.0, Avg 3.86, Std 3.3912239678322633, Avg % of duplicate genes across participants 0.0962
27.95
Generation 4: Min 0.0, Max 15.0, Avg 5.26, Std 3.2391974314635408, Avg % of duplicate genes across participants 0.0974
36.56
Generation 5: Min 0.0, Max 17.0, Avg 7.03, Std 4.179605244517716, Avg % of duplicate genes across participants 0.0967
39.58
Generation 6: Min 0.0, Max 22.0, Avg 9.79, Std 5.1211229237346, Avg % of duplicate genes across participants 0.0934
47.71
Generation 7: Min 0.0, Max 24.0, Avg 12.79, Std 5.372699507696295, Avg % of duplicate genes across participants 0.0926

# Bilingualism

### save languages

In [545]:
population1 = toolbox.population(n=100)
population2 = toolbox.population(n=100)


""" Predefined languages evolved from above code
Select one of the pairs
"""
# pair 1
# predefined_language = {0: 'bi', 1: 'tgg', 2: 'a', 3: 'v', 4: 've', 5: 'ka', 6: 'i', 7: 'fm', 8: 'gw', 9: 'ha', 10: 'eol', 11: 'rqi', 12: 'r', 13: 'zxw', 14: 'ab', 15: 'fnl', 16: 'mju', 17: 'w', 18: 'nhm', 19: 'qs', 20: 'dgl', 21: 'n', 22: 'zou', 23: 'fo', 24: 'mmx', 25: 'u', 26: 'zh', 27: 'qi', 28: 'lrv', 29: 'vtg', 30: 'bjj', 31: 'vw', 32: 'hgk', 33: 'ttm', 34: 'vo', 35: 'fk', 36: 'up', 37: 'end', 38: 'fd', 39: 'zwl', 40: 'sm', 41: 'b', 42: 'wt', 43: 'qtj', 44: 'z', 45: 'yl', 46: 'zms', 47: 'fos', 48: 'kp', 49: 'y', 50: 'akh', 51: 'j', 52: 'hcd', 53: 'ed', 54: 'mw', 55: 'ow', 56: 'rl', 57: 'xp', 58: 'xqx', 59: 'yy', 60: 'lll', 61: 'ys', 62: 'ifh', 63: 'p', 64: 'g', 65: 'ob', 66: 'ur', 67: 'k', 68: 'acz', 69: 'kwt', 70: 'qt', 71: 'ax', 72: 'pn', 73: 'ly', 74: 'oxu', 75: 'ru', 76: 'c', 77: 'les', 78: 'vso', 79: 'pjy', 80: 'qpm', 81: 'scr', 82: 'hdh', 83: 'm', 84: 'gnj', 85: 'ut', 86: 'bu', 87: 'uu', 88: 'yf', 89: 's', 90: 'jdy', 91: 'wte', 92: 'ufo', 93: 'yw', 94: 'zd', 95: 'l', 96: 'd', 97: 'na', 98: 'iik', 99: 'qmj'}
# predefined_language2 = {0: 'xg', 1: 'os', 2: 'mfz', 3: 'it', 4: 'yy', 5: 'bzd', 6: 'nh', 7: 'ihg', 8: 'azu', 9: 'uul', 10: 'wem', 11: 'qwc', 12: 'ik', 13: 'dfb', 14: 'sdl', 15: 'hn', 16: 'vo', 17: 'ug', 18: 'vte', 19: 'iw', 20: 'sf', 21: 'vt', 22: 'g', 23: 'b', 24: 'pa', 25: 'god', 26: 'vf', 27: 'y', 28: 'ud', 29: 'db', 30: 'ax', 31: 'n', 32: 'in', 33: 'ztu', 34: 'syu', 35: 'd', 36: 'ic', 37: 'qem', 38: 'rux', 39: 'wt', 40: 'hp', 41: 'sh', 42: 'ak', 43: 'dsh', 44: 'bur', 45: 'jt', 46: 'j', 47: 'cqw', 48: 'tdk', 49: 'sy', 50: 'z', 51: 'm', 52: 'kvm', 53: 'old', 54: 'bqz', 55: 'crw', 56: 'eys', 57: 'h', 58: 'vh', 59: 'bk', 60: 'fzo', 61: 'u', 62: 'jdu', 63: 's', 64: 'npx', 65: 'xs', 66: 'ob', 67: 'i', 68: 'asx', 69: 'tdn', 70: 'th', 71: 'k', 72: 'vrs', 73: 'ozq', 74: 'bp', 75: 'ih', 76: 'rm', 77: 'no', 78: 'sr', 79: 'qyk', 80: 'yqq', 81: 'em', 82: 'xb', 83: 'tq', 84: 'c', 85: 'ox', 86: 'geu', 87: 'yu', 88: 'yc', 89: 'td', 90: 'qa', 91: 'r', 92: 'dd', 93: 'e', 94: 'od', 95: 'xj', 96: 'am', 97: 'wnw', 98: 'tl', 99: 'buf'}

# pair 2
# predefined_language = {0: 'bq', 1: 'b', 2: 'kpi', 3: 'yvm', 4: 'bcf', 5: 'rus', 6: 'h', 7: 'dv', 8: 'uh', 9: 'qae', 10: 'owx', 11: 'tu', 12: 'hxl', 13: 'e', 14: 'oe', 15: 'rff', 16: 'koo', 17: 'tzz', 18: 'hp', 19: 'cvg', 20: 'ry', 21: 'ozt', 22: 'rnt', 23: 'vdx', 24: 'ar', 25: 'ofm', 26: 'cyp', 27: 'gfl', 28: 'rgj', 29: 'tc', 30: 'pcc', 31: 'jcr', 32: 'ugn', 33: 'oa', 34: 'io', 35: 'q', 36: 'dl', 37: 'xfy', 38: 'cgd', 39: 'lw', 40: 's', 41: 'a', 42: 'f', 43: 'm', 44: 'ia', 45: 'rm', 46: 'ke', 47: 'hn', 48: 'zrn', 49: 'r', 50: 'ytj', 51: 'u', 52: 't', 53: 'ssv', 54: 'n', 55: 'vi', 56: 'vm', 57: 'wkr', 58: 'k', 59: 'ong', 60: 'th', 61: 'uw', 62: 'l', 63: 'c', 64: 'bbq', 65: 'rc', 66: 'ddl', 67: 'ly', 68: 'xvf', 69: 'ktd', 70: 'yw', 71: 'xne', 72: 'tms', 73: 'i', 74: 'ot', 75: 'gp', 76: 'wx', 77: 'dwg', 78: 'kml', 79: 'bdf', 80: 'yk', 81: 'ux', 82: 'sj', 83: 'fx', 84: 'yt', 85: 'kf', 86: 'mp', 87: 'fe', 88: 'w', 89: 'yd', 90: 'y', 91: 'j', 92: 'md', 93: 'x', 94: 'gwc', 95: 'cns', 96: 'of', 97: 'zp', 98: 'zl', 99: 'ogw'} 
# predefined_language2 = {0: 'pdy', 1: 'p', 2: 'bnr', 3: 'rjs', 4: 'qnt', 5: 'krz', 6: 'rru', 7: 'j', 8: 'yol', 9: 'du', 10: 's', 11: 'ihm', 12: 'qw', 13: 'fm', 14: 'fc', 15: 'fq', 16: 'bd', 17: 'zp', 18: 'me', 19: 'mb', 20: 't', 21: 'ba', 22: 'h', 23: 'xwu', 24: 'w', 25: 'xjf', 26: 'ie', 27: 'tsa', 28: 'dh', 29: 'fwk', 30: 'zrm', 31: 'tff', 32: 'ql', 33: 'hzh', 34: 'rnj', 35: 'lsc', 36: 'u', 37: 'hwy', 38: 'zdo', 39: 'y', 40: 'yt', 41: 'on', 42: 'bh', 43: 'xed', 44: 'ni', 45: 'dwl', 46: 'd', 47: 'x', 48: 'aqu', 49: 'jqq', 50: 'q', 51: 'rg', 52: 'olc', 53: 'g', 54: 'xci', 55: 'rph', 56: 'n', 57: 'lu', 58: 'jd', 59: 'ra', 60: 're', 61: 'k', 62: 'sau', 63: 'md', 64: 'pgg', 65: 'afv', 66: 'r', 67: 'ccn', 68: 'kc', 69: 'oz', 70: 'ncr', 71: 'gqd', 72: 'yvc', 73: 'e', 74: 'kv', 75: 'co', 76: 'ss', 77: 'pr', 78: 'o', 79: 'rup', 80: 'xq', 81: 'xau', 82: 'agc', 83: 'c', 84: 'wmf', 85: 'pi', 86: 'evm', 87: 'it', 88: 'uk', 89: 'uep', 90: 'cm', 91: 'phn', 92: 'jqs', 93: 'un', 94: 'dm', 95: 'kbr', 96: 'wpm', 97: 'a', 98: 'rt', 99: 'qga'} 

# pair 3
# predefined_language = {0: 'ic', 1: 'd', 2: 'xes', 3: 'qi', 4: 'u', 5: 'daj', 6: 'xl', 7: 'y', 8: 'rci', 9: 'gpk', 10: 'ew', 11: 'es', 12: 'e', 13: 'ixa', 14: 'js', 15: 'cn', 16: 'fvp', 17: 'vp', 18: 'fdv', 19: 'bg', 20: 'qeh', 21: 'rem', 22: 'fsr', 23: 'bwl', 24: 'qb', 25: 'pu', 26: 'bwv', 27: 'llk', 28: 'w', 29: 'em', 30: 'zfr', 31: 'jsb', 32: 't', 33: 'fnf', 34: 'vb', 35: 'ji', 36: 'pr', 37: 'zyv', 38: 'wve', 39: 'h', 40: 'nm', 41: 'p', 42: 'vi', 43: 'sa', 44: 'eli', 45: 'uyi', 46: 'lgl', 47: 'do', 48: 'ck', 49: 'j', 50: 'r', 51: 'gx', 52: 'lai', 53: 'rwy', 54: 'sd', 55: 'z', 56: 'ceo', 57: 'sxc', 58: 'rx', 59: 'bc', 60: 'pfq', 61: 'cz', 62: 'zn', 63: 'b', 64: 'trl', 65: 'bii', 66: 'sm', 67: 'ph', 68: 'ja', 69: 'm', 70: 'ep', 71: 's', 72: 'hb', 73: 'zyz', 74: 'wy', 75: 'v', 76: 'c', 77: 'jy', 78: 'kgs', 79: 'ipf', 80: 'f', 81: 'xy', 82: 'hx', 83: 'cdw', 84: 'x', 85: 'ze', 86: 'pun', 87: 'g', 88: 'whg', 89: 'k', 90: 'owc', 91: 'uq', 92: 'pv', 93: 'tpf', 94: 'rwz', 95: 'q', 96: 'iy', 97: 'ss', 98: 'ga', 99: 'bo'} 
# predefined_language2 = {0: 'eb', 1: 'rll', 2: 'bo', 3: 'ce', 4: 'kr', 5: 'm', 6: 'jpm', 7: 'ye', 8: 'nwa', 9: 'zp', 10: 'rq', 11: 's', 12: 'wo', 13: 'wnt', 14: 'kht', 15: 'ju', 16: 'h', 17: 't', 18: 'khu', 19: 'jvq', 20: 'kx', 21: 'o', 22: 'gzm', 23: 'qh', 24: 'y', 25: 'b', 26: 'lvs', 27: 'id', 28: 'ib', 29: 'yvv', 30: 'piz', 31: 'tj', 32: 'bvd', 33: 'ynb', 34: 'qz', 35: 'g', 36: 'mxg', 37: 'elz', 38: 'sq', 39: 'gtr', 40: 'dj', 41: 'egs', 42: 'mr', 43: 'ts', 44: 'cse', 45: 'vp', 46: 'w', 47: 'ek', 48: 'azd', 49: 'l', 50: 'in', 51: 'odr', 52: 'bw', 53: 'omm', 54: 'x', 55: 'gc', 56: 'pm', 57: 'fpz', 58: 'tbj', 59: 'sa', 60: 'i', 61: 'tgk', 62: 'se', 63: 'qub', 64: 'mx', 65: 'peb', 66: 'eq', 67: 'pu', 68: 'xvq', 69: 'bte', 70: 'tr', 71: 'fn', 72: 'z', 73: 'abk', 74: 'ys', 75: 'na', 76: 'yrf', 77: 'k', 78: 'fte', 79: 'iq', 80: 'osf', 81: 'd', 82: 'icd', 83: 'oas', 84: 'q', 85: 'zzu', 86: 'j', 87: 'xz', 88: 'hrs', 89: 'jid', 90: 'isr', 91: 'xtg', 92: 'eqv', 93: 'zil', 94: 'c', 95: 'ft', 96: 'wsy', 97: 'xun', 98: 'zee', 99: 'pgx'} 

# pair 4
# predefined_language = {0: 'yc', 1: 'oih', 2: 'h', 3: 'k', 4: 'f', 5: 'ksn', 6: 'kuw', 7: 'coc', 8: 'qz', 9: 'qyt', 10: 'ef', 11: 'er', 12: 'dc', 13: 'vrv', 14: 'c', 15: 'jyl', 16: 'u', 17: 'xqc', 18: 'sqk', 19: 'mio', 20: 'ke', 21: 'i', 22: 'gsa', 23: 'ou', 24: 'uc', 25: 'jzt', 26: 'q', 27: 'wiw', 28: 'yad', 29: 'bxi', 30: 'gkp', 31: 'uzz', 32: 'wwp', 33: 'my', 34: 'or', 35: 'qu', 36: 'a', 37: 'wph', 38: 'lm', 39: 'no', 40: 'n', 41: 'wp', 42: 'dlx', 43: 'wj', 44: 'jp', 45: 'ocr', 46: 'gi', 47: 'ycx', 48: 'zoc', 49: 'sb', 50: 'rzk', 51: 'e', 52: 'wm', 53: 'foc', 54: 'bo', 55: 's', 56: 'rsf', 57: 'tb', 58: 'bg', 59: 'x', 60: 'wok', 61: 'ffm', 62: 'yl', 63: 'fcq', 64: 'tu', 65: 'wb', 66: 'wpd', 67: 'sh', 68: 'r', 69: 'rk', 70: 'bdd', 71: 'dr', 72: 'tgv', 73: 'mlh', 74: 'bio', 75: 'iu', 76: 'ka', 77: 'bwr', 78: 'gr', 79: 'mhm', 80: 'ng', 81: 'iz', 82: 'uk', 83: 'hst', 84: 'ojz', 85: 'w', 86: 'm', 87: 'vvh', 88: 'hf', 89: 'yk', 90: 'v', 91: 'vct', 92: 'ej', 93: 'hw', 94: 'xvc', 95: 'lor', 96: 'uu', 97: 'pta', 98: 't', 99: 'l'} 
# predefined_language2 = {0: 'h', 1: 'ic', 2: 'cxj', 3: 'gwd', 4: 'p', 5: 'ns', 6: 'nn', 7: 'bwv', 8: 'jr', 9: 'ik', 10: 'ime', 11: 'tb', 12: 'sqq', 13: 'tvo', 14: 'lh', 15: 'k', 16: 'yo', 17: 'dz', 18: 'npc', 19: 'sik', 20: 'vh', 21: 'bx', 22: 'w', 23: 'syx', 24: 'ckb', 25: 'kgf', 26: 'm', 27: 'khy', 28: 'ig', 29: 'ag', 30: 'tp', 31: 'ek', 32: 's', 33: 'qmj', 34: 'qw', 35: 'r', 36: 'zt', 37: 'oj', 38: 'n', 39: 'ne', 40: 'vtx', 41: 'fu', 42: 'wkd', 43: 'jxe', 44: 'sqg', 45: 'x', 46: 'd', 47: 'kj', 48: 'ue', 49: 'jf', 50: 'zvc', 51: 'itk', 52: 'ni', 53: 'gz', 54: 'tmb', 55: 'tr', 56: 'fe', 57: 'e', 58: 'dhr', 59: 'qgc', 60: 'ql', 61: 'ur', 62: 'c', 63: 'twn', 64: 'jtn', 65: 'br', 66: 'nog', 67: 'z', 68: 'i', 69: 'bmn', 70: 'goq', 71: 'xd', 72: 'msr', 73: 'us', 74: 'ts', 75: 'mct', 76: 'mqa', 77: 'v', 78: 'ldu', 79: 'fsm', 80: 'yoe', 81: 'j', 82: 'lbl', 83: 'pkr', 84: 'cw', 85: 'ipg', 86: 'urn', 87: 'ob', 88: 'tmm', 89: 'bd', 90: 'gml', 91: 'sr', 92: 'gi', 93: 'mae', 94: 'wld', 95: 'sym', 96: 'g', 97: 'ksa', 98: 'lx', 99: 'fa'} 

# pair 5
# predefined_language = {0: 'j', 1: 'rmp', 2: 'pxl', 3: 'zbg', 4: 'ux', 5: 'sdj', 6: 't', 7: 'ru', 8: 'tf', 9: 'wj', 10: 'vg', 11: 'cb', 12: 'ir', 13: 'r', 14: 'pjw', 15: 'ryh', 16: 'acy', 17: 'do', 18: 'icd', 19: 'o', 20: 'sgq', 21: 'xt', 22: 'vvx', 23: 'wir', 24: 'om', 25: 'fa', 26: 'wp', 27: 'x', 28: 'ddf', 29: 'ext', 30: 'pz', 31: 'gfj', 32: 'qs', 33: 'p', 34: 'lc', 35: 'mw', 36: 'onj', 37: 'xi', 38: 'bx', 39: 'hmk', 40: 'dk', 41: 'sdn', 42: 'gp', 43: 'km', 44: 's', 45: 'wi', 46: 'xfw', 47: 'psc', 48: 'tfu', 49: 'per', 50: 'jy', 51: 'oas', 52: 'usx', 53: 'tg', 54: 'em', 55: 'qkl', 56: 'ao', 57: 'qh', 58: 'k', 59: 'v', 60: 'fx', 61: 'dm', 62: 'yq', 63: 'bp', 64: 'od', 65: 'lxm', 66: 'zdm', 67: 'uq', 68: 'aik', 69: 'br', 70: 'aa', 71: 'gtj', 72: 'm', 73: 'vhk', 74: 'iob', 75: 'st', 76: 'bnf', 77: 'hfv', 78: 'phi', 79: 'go', 80: 'no', 81: 'cf', 82: 'gn', 83: 'vin', 84: 'y', 85: 'cjv', 86: 'cqe', 87: 'dy', 88: 'gd', 89: 'u', 90: 'l', 91: 'q', 92: 'xhh', 93: 'dft', 94: 'os', 95: 'iuv', 96: 'le', 97: 'cz', 98: 'g', 99: 'a'} 
# predefined_language2 = {0: 'emv', 1: 'tp', 2: 'vb', 3: 'uo', 4: 'xt', 5: 'mo', 6: 'v', 7: 'ic', 8: 'xm', 9: 'r', 10: 'pwk', 11: 'e', 12: 'anl', 13: 'l', 14: 'ch', 15: 'mm', 16: 'vvk', 17: 'n', 18: 'zj', 19: 'dg', 20: 'lr', 21: 'ju', 22: 'wmo', 23: 'iv', 24: 'gna', 25: 'jj', 26: 'zho', 27: 'ucb', 28: 'gom', 29: 's', 30: 'm', 31: 'odf', 32: 'br', 33: 'qa', 34: 'c', 35: 'dx', 36: 'rf', 37: 'gp', 38: 'i', 39: 'ywh', 40: 'w', 41: 'zp', 42: 'f', 43: 'a', 44: 'mif', 45: 'to', 46: 'mhx', 47: 'did', 48: 'bg', 49: 'uuf', 50: 'wca', 51: 'ki', 52: 'qc', 53: 'ux', 54: 'uva', 55: 'qp', 56: 'p', 57: 'glg', 58: 'ybi', 59: 'iaz', 60: 'cn', 61: 'zce', 62: 'zh', 63: 'xtv', 64: 'wbu', 65: 'x', 66: 'h', 67: 'mbj', 68: 'bd', 69: 'gl', 70: 'k', 71: 'd', 72: 'yv', 73: 'u', 74: 'lw', 75: 'nif', 76: 'z', 77: 'ajb', 78: 'ixp', 79: 'tlh', 80: 'qf', 81: 'kh', 82: 'wf', 83: 'ucc', 84: 'ncy', 85: 'xr', 86: 'fdn', 87: 'ui', 88: 'nbr', 89: 'tz', 90: 'ec', 91: 'id', 92: 'en', 93: 'bm', 94: 'g', 95: 'kx', 96: 't', 97: 'fzg', 98: 'jye', 99: 'gyl'} 

# pair 6
# predefined_language = {0: 'u', 1: 'j', 2: 'fr', 3: 'xzv', 4: 'o', 5: 'b', 6: 'hex', 7: 'hfg', 8: 'xp', 9: 'dr', 10: 'rjx', 11: 'rxa', 12: 'kxl', 13: 'y', 14: 'jc', 15: 'psl', 16: 'uxw', 17: 'xhc', 18: 'tko', 19: 'ka', 20: 'jch', 21: 'sa', 22: 'vr', 23: 'gbm', 24: 'nh', 25: 'aby', 26: 'h', 27: 'wyr', 28: 'fxp', 29: 'fjv', 30: 's', 31: 'jt', 32: 'uuo', 33: 'k', 34: 'ot', 35: 'cql', 36: 'nm', 37: 't', 38: 'nl', 39: 'ucq', 40: 'mr', 41: 'n', 42: 'frk', 43: 'nhx', 44: 'qk', 45: 'qfy', 46: 'ue', 47: 'ccr', 48: 'v', 49: 'ns', 50: 'hq', 51: 'aj', 52: 'jhs', 53: 'p', 54: 'dt', 55: 'lum', 56: 'tof', 57: 'ig', 58: 'ghv', 59: 'gs', 60: 'zc', 61: 'rp', 62: 'cpt', 63: 'uf', 64: 'uo', 65: 'hw', 66: 'xte', 67: 'iq', 68: 'f', 69: 'gel', 70: 'pw', 71: 'pe', 72: 'dfx', 73: 'bf', 74: 'ym', 75: 'qkq', 76: 'fcz', 77: 'gq', 78: 'xa', 79: 'lpi', 80: 'pn', 81: 'yeu', 82: 'a', 83: 'ho', 84: 'fyi', 85: 'hm', 86: 'l', 87: 'jjy', 88: 'r', 89: 'jzz', 90: 'fy', 91: 'lh', 92: 'xtd', 93: 'ua', 94: 'w', 95: 'kd', 96: 'onc', 97: 'sw', 98: 'err', 99: 'crq'} 
# predefined_language2 = {0: 'l', 1: 'ud', 2: 'lsd', 3: 'f', 4: 'xsm', 5: 'uxv', 6: 'chf', 7: 'h', 8: 'yzi', 9: 'szk', 10: 'ps', 11: 'bsx', 12: 'u', 13: 'dnq', 14: 'oz', 15: 'vpx', 16: 'bi', 17: 'mbq', 18: 'xd', 19: 'ra', 20: 'qu', 21: 'x', 22: 'tz', 23: 'ejd', 24: 'jtw', 25: 'hcn', 26: 'ypd', 27: 'zq', 28: 'vkm', 29: 'b', 30: 'lnb', 31: 'vnf', 32: 'qvn', 33: 'kv', 34: 'ecx', 35: 'rx', 36: 'v', 37: 'uli', 38: 'ph', 39: 'aa', 40: 'piy', 41: 'ekv', 42: 'afc', 43: 'mo', 44: 'fs', 45: 'cs', 46: 'ycb', 47: 'ski', 48: 'bq', 49: 'fa', 50: 'in', 51: 'con', 52: 'yfx', 53: 'vsl', 54: 'lia', 55: 'wxj', 56: 'vg', 57: 'spk', 58: 'pt', 59: 'w', 60: 'zu', 61: 'hyp', 62: 'n', 63: 'jyl', 64: 'bo', 65: 'tm', 66: 'odl', 67: 'p', 68: 'rp', 69: 't', 70: 'eia', 71: 'nlq', 72: 'q', 73: 'a', 74: 'ap', 75: 'uc', 76: 'cn', 77: 'd', 78: 'yl', 79: 'gvt', 80: 'qz', 81: 'lmq', 82: 'oa', 83: 'cds', 84: 'sk', 85: 'lu', 86: 'un', 87: 'ny', 88: 'z', 89: 'jz', 90: 'xqd', 91: 'ow', 92: 'y', 93: 'swo', 94: 'qo', 95: 'lef', 96: 'ori', 97: 'by', 98: 'lw', 99: 'rei'} 

# pair 7
# predefined_language = {0: 'mil', 1: 'tr', 2: 'oyz', 3: 'okf', 4: 'oxn', 5: 'dfo', 6: 'qat', 7: 'hu', 8: 'kl', 9: 'q', 10: 'a', 11: 'pz', 12: 'fdk', 13: 'rx', 14: 'bi', 15: 'nu', 16: 'e', 17: 'mf', 18: 'rl', 19: 'anx', 20: 'wqd', 21: 'asa', 22: 'pxl', 23: 'fyv', 24: 'wuk', 25: 'hjv', 26: 'unt', 27: 'h', 28: 'hv', 29: 'xf', 30: 'j', 31: 'ra', 32: 'd', 33: 'ea', 34: 'ds', 35: 'wr', 36: 'eco', 37: 't', 38: 'fk', 39: 'dm', 40: 'ut', 41: 'jn', 42: 'ez', 43: 'zkc', 44: 'fg', 45: 'fqc', 46: 'ro', 47: 's', 48: 'vlb', 49: 'ug', 50: 'rb', 51: 'ld', 52: 'nr', 53: 'fl', 54: 'ul', 55: 'xzb', 56: 'zb', 57: 'yss', 58: 'nju', 59: 'gr', 60: 'y', 61: 'zy', 62: 'isx', 63: 'fnu', 64: 'mmx', 65: 'yg', 66: 'x', 67: 'cbi', 68: 'fof', 69: 'f', 70: 'ee', 71: 'gbf', 72: 'k', 73: 'qyq', 74: 'r', 75: 'saf', 76: 'hqx', 77: 'kbx', 78: 'kvf', 79: 'jfe', 80: 'w', 81: 'dt', 82: 'fp', 83: 'yyu', 84: 'bf', 85: 'vhj', 86: 'qm', 87: 'fwb', 88: 'bmi', 89: 'kux', 90: 'im', 91: 'sy', 92: 'lvy', 93: 'fn', 94: 'ekx', 95: 'tlm', 96: 'vgl', 97: 'yr', 98: 'iz', 99: 'gz'} 
# predefined_language2 = {0: 'uyu', 1: 'lv', 2: 'lqq', 3: 'ztt', 4: 'm', 5: 'qj', 6: 'rok', 7: 'd', 8: 'neu', 9: 'u', 10: 'h', 11: 'q', 12: 'pce', 13: 'fmf', 14: 'lf', 15: 'hoj', 16: 'o', 17: 'vif', 18: 'ha', 19: 'p', 20: 'kin', 21: 'ell', 22: 'ivi', 23: 'idk', 24: 'jq', 25: 'qk', 26: 'ovh', 27: 'j', 28: 'c', 29: 'ln', 30: 'x', 31: 'n', 32: 'fpk', 33: 'nyf', 34: 'rc', 35: 'dw', 36: 'i', 37: 'xd', 38: 'k', 39: 'b', 40: 'ii', 41: 'asi', 42: 'jy', 43: 'ab', 44: 'hyx', 45: 'kry', 46: 'z', 47: 'qqg', 48: 'rk', 49: 'pgm', 50: 'hg', 51: 'lds', 52: 'zph', 53: 'w', 54: 'tth', 55: 'tdf', 56: 's', 57: 'kgy', 58: 'yi', 59: 'wqo', 60: 'uf', 61: 'tn', 62: 'dmz', 63: 'ry', 64: 'yw', 65: 'rtn', 66: 'l', 67: 'cq', 68: 'zb', 69: 'a', 70: 'frf', 71: 'ska', 72: 'eqc', 73: 'nv', 74: 'bda', 75: 'mfp', 76: 'pqn', 77: 'ns', 78: 'cks', 79: 'nhg', 80: 'y', 81: 'hx', 82: 'ivr', 83: 'zhr', 84: 't', 85: 'po', 86: 'vz', 87: 'zj', 88: 'wiu', 89: 'jvt', 90: 'yd', 91: 'g', 92: 'uav', 93: 'vsv', 94: 'cqg', 95: 'tp', 96: 'e', 97: 'tzo', 98: 'll', 99: 'dxe'} 

# pair 8
# predefined_language = {0: 'tnc', 1: 'h', 2: 'fs', 3: 'csk', 4: 'uaa', 5: 'cb', 6: 'lc', 7: 'q', 8: 'ux', 9: 'rt', 10: 'chg', 11: 'zxx', 12: 'zyz', 13: 'lwd', 14: 'vjh', 15: 'k', 16: 'm', 17: 'jrn', 18: 'iu', 19: 'r', 20: 'pa', 21: 'mer', 22: 'qw', 23: 'hlu', 24: 'yi', 25: 'mc', 26: 'ic', 27: 'w', 28: 'bl', 29: 'bi', 30: 'pi', 31: 'tv', 32: 'ph', 33: 'ybf', 34: 'et', 35: 'pue', 36: 'zj', 37: 'n', 38: 'vl', 39: 'l', 40: 't', 41: 'ys', 42: 'iyr', 43: 'sfe', 44: 's', 45: 'kgl', 46: 'z', 47: 'mf', 48: 'hpx', 49: 'g', 50: 'u', 51: 'kv', 52: 'dm', 53: 'b', 54: 'hdm', 55: 'zl', 56: 'ixd', 57: 'ai', 58: 'bcu', 59: 'v', 60: 'wjk', 61: 'fc', 62: 'bx', 63: 'wgu', 64: 'ev', 65: 'sn', 66: 'fv', 67: 'iwe', 68: 'kkt', 69: 'j', 70: 'xtm', 71: 'am', 72: 'so', 73: 'sq', 74: 'lx', 75: 'pnn', 76: 'opg', 77: 'ui', 78: 'gci', 79: 'o', 80: 'emm', 81: 'bp', 82: 'cse', 83: 'fpd', 84: 'hbz', 85: 'i', 86: 'goy', 87: 'jm', 88: 'iq', 89: 'sh', 90: 'bhc', 91: 'myw', 92: 'hsn', 93: 'huf', 94: 'sml', 95: 'arx', 96: 'p', 97: 'ct', 98: 'tdp', 99: 'wod'} 
# predefined_language2 = {0: 'rq', 1: 'n', 2: 'qe', 3: 'wyy', 4: 'op', 5: 'ear', 6: 'cpd', 7: 'clp', 8: 'ho', 9: 'any', 10: 'jul', 11: 'hm', 12: 'x', 13: 'nuv', 14: 'zrm', 15: 'hbk', 16: 'no', 17: 'ms', 18: 'pp', 19: 'led', 20: 'xgv', 21: 'c', 22: 'rh', 23: 'h', 24: 'dc', 25: 'ubn', 26: 'aax', 27: 'jv', 28: 'jay', 29: 'z', 30: 'tud', 31: 'pi', 32: 'zhp', 33: 'au', 34: 'ft', 35: 'ow', 36: 'og', 37: 'bak', 38: 'spv', 39: 'izx', 40: 'hv', 41: 'nq', 42: 'hhf', 43: 'zws', 44: 'ryu', 45: 'u', 46: 'due', 47: 'gho', 48: 'j', 49: 'rlk', 50: 'rlt', 51: 'yqh', 52: 'cvo', 53: 'zmb', 54: 'ctg', 55: 'mp', 56: 'ka', 57: 'my', 58: 'f', 59: 'fvw', 60: 'xn', 61: 'm', 62: 'lqm', 63: 'aka', 64: 'yni', 65: 'bm', 66: 'w', 67: 'bb', 68: 'izy', 69: 'qgg', 70: 'ax', 71: 'vsy', 72: 'uyb', 73: 'g', 74: 'kny', 75: 'y', 76: 'fed', 77: 'gh', 78: 's', 79: 'le', 80: 'vz', 81: 'i', 82: 'cu', 83: 'cr', 84: 'izd', 85: 'zfr', 86: 'dho', 87: 'dvx', 88: 'at', 89: 'ivk', 90: 'szk', 91: 'r', 92: 'dw', 93: 'ru', 94: 'mpb', 95: 'bnj', 96: 'hf', 97: 'ly', 98: 'jbs', 99: 'ei'} 

# pair 9
# predefined_language = {0: 'sw', 1: 't', 2: 'zsr', 3: 'aol', 4: 'gn', 5: 'pz', 6: 'tlk', 7: 'rgm', 8: 'rga', 9: 'sz', 10: 'rsv', 11: 'oix', 12: 'f', 13: 'an', 14: 'ita', 15: 'fv', 16: 'ljs', 17: 'qpq', 18: 'lvu', 19: 'z', 20: 'kv', 21: 'r', 22: 'mc', 23: 'nik', 24: 'uyu', 25: 'd', 26: 'jhv', 27: 'x', 28: 'uy', 29: 'ky', 30: 'fe', 31: 's', 32: 'gfh', 33: 'pmq', 34: 'ld', 35: 'wln', 36: 'c', 37: 'om', 38: 'jj', 39: 'nm', 40: 'gr', 41: 'sg', 42: 'k', 43: 'vc', 44: 'xm', 45: 'xhl', 46: 'nsk', 47: 'qef', 48: 'mcv', 49: 'dtg', 50: 'cu', 51: 'xwk', 52: 'am', 53: 'b', 54: 'jjw', 55: 'yjv', 56: 'cch', 57: 'je', 58: 'yr', 59: 'sc', 60: 'q', 61: 'w', 62: 'ci', 63: 'qf', 64: 'tp', 65: 'g', 66: 'nu', 67: 'lij', 68: 'gcq', 69: 'rst', 70: 'xq', 71: 'vl', 72: 'p', 73: 'cw', 74: 'us', 75: 'qh', 76: 'xbf', 77: 'ifu', 78: 'px', 79: 'gfj', 80: 'cjl', 81: 'n', 82: 'gb', 83: 'agj', 84: 'et', 85: 'bcj', 86: 'gy', 87: 'fw', 88: 'ohm', 89: 'hsc', 90: 'el', 91: 'es', 92: 'ydz', 93: 'o', 94: 'l', 95: 'wu', 96: 'u', 97: 'yut', 98: 'm', 99: 'ar'} 
# predefined_language2 = {0: 's', 1: 'ew', 2: 'q', 3: 'qjp', 4: 'ufe', 5: 'jq', 6: 'nq', 7: 'hl', 8: 'tdp', 9: 'gz', 10: 'se', 11: 'u', 12: 'txc', 13: 'vb', 14: 'v', 15: 'ct', 16: 'tc', 17: 'mdo', 18: 'l', 19: 'dws', 20: 'bn', 21: 'fs', 22: 'hy', 23: 'jmf', 24: 'cyq', 25: 'r', 26: 'ccz', 27: 'xyw', 28: 'cma', 29: 'kvn', 30: 'kkj', 31: 'lab', 32: 'sy', 33: 'vl', 34: 'n', 35: 'qav', 36: 'ems', 37: 'yhw', 38: 'uid', 39: 'fir', 40: 'of', 41: 'pnw', 42: 'i', 43: 'z', 44: 'y', 45: 'hf', 46: 'k', 47: 'orc', 48: 'cqc', 49: 'mv', 50: 'jxv', 51: 'pd', 52: 'g', 53: 'dyf', 54: 'wi', 55: 'dsu', 56: 'asl', 57: 'pyj', 58: 'brs', 59: 'dfp', 60: 'x', 61: 'c', 62: 'oaz', 63: 'ww', 64: 't', 65: 'imb', 66: 'grg', 67: 'o', 68: 'lk', 69: 'cv', 70: 'xwy', 71: 'tl', 72: 'jj', 73: 'pr', 74: 'xsu', 75: 'p', 76: 'du', 77: 'cw', 78: 'kfv', 79: 'vbv', 80: 'hyv', 81: 'qu', 82: 'cbz', 83: 'in', 84: 'rqy', 85: 'mbz', 86: 'kh', 87: 'tg', 88: 'rh', 89: 'dt', 90: 'nz', 91: 'jso', 92: 'uw', 93: 'gus', 94: 'jxo', 95: 'pdl', 96: 'bnj', 97: 'sbh', 98: 'io', 99: 'b'} 

# pair 10
predefined_language = {0: 'qra', 1: 'dfe', 2: 'oi', 3: 'bdx', 4: 'juy', 5: 'tj', 6: 'de', 7: 'vw', 8: 'n', 9: 't', 10: 'cdd', 11: 'xgy', 12: 'w', 13: 'tha', 14: 'ch', 15: 'zc', 16: 'm', 17: 'vcx', 18: 'dyt', 19: 's', 20: 'xiu', 21: 'i', 22: 'gb', 23: 'rcd', 24: 'dsu', 25: 'lat', 26: 'blg', 27: 'alc', 28: 'gyi', 29: 'vml', 30: 'jtj', 31: 'py', 32: 'hc', 33: 'ujt', 34: 'mx', 35: 'fhg', 36: 'jgq', 37: 'o', 38: 'odz', 39: 'xgl', 40: 'wb', 41: 'cx', 42: 'p', 43: 'lh', 44: 'amc', 45: 'ejd', 46: 'e', 47: 'h', 48: 'oll', 49: 'zho', 50: 'ei', 51: 'uyq', 52: 'cre', 53: 'dro', 54: 'f', 55: 'pbd', 56: 'sc', 57: 'ygk', 58: 'ng', 59: 'b', 60: 'r', 61: 'c', 62: 'q', 63: 'phg', 64: 'lo', 65: 'fwd', 66: 'x', 67: 'px', 68: 'as', 69: 'zd', 70: 'yl', 71: 'fxh', 72: 'oy', 73: 'awc', 74: 'ne', 75: 'wqw', 76: 'g', 77: 'eud', 78: 'eqa', 79: 'v', 80: 're', 81: 'agh', 82: 'l', 83: 'wjs', 84: 'yx', 85: 'k', 86: 'd', 87: 'bx', 88: 'oz', 89: 'yep', 90: 'y', 91: 'iwo', 92: 'jw', 93: 'ck', 94: 'ry', 95: 'ugn', 96: 'sem', 97: 'vyd', 98: 'pq', 99: 'hme'} 
predefined_language2 = {0: 'ft', 1: 'zg', 2: 'hvy', 3: 'wx', 4: 'qx', 5: 'tg', 6: 'ths', 7: 'df', 8: 'vpw', 9: 'uyh', 10: 'mh', 11: 'j', 12: 'yvo', 13: 'x', 14: 'wfx', 15: 'zfa', 16: 'ne', 17: 'ku', 18: 'pg', 19: 'm', 20: 'yl', 21: 'ius', 22: 'uv', 23: 'zw', 24: 'od', 25: 'ea', 26: 'jr', 27: 'clh', 28: 'eyv', 29: 'myg', 30: 'p', 31: 'ege', 32: 'dnl', 33: 'khm', 34: 'aa', 35: 'ho', 36: 'dq', 37: 'qf', 38: 'eho', 39: 'f', 40: 'npv', 41: 'ag', 42: 'e', 43: 'ms', 44: 'ep', 45: 'q', 46: 'z', 47: 'a', 48: 'py', 49: 'fw', 50: 'hhp', 51: 'jhx', 52: 'xr', 53: 'xtu', 54: 'hl', 55: 'fhl', 56: 'r', 57: 'jhz', 58: 'nlv', 59: 'uil', 60: 'y', 61: 'v', 62: 'w', 63: 'niu', 64: 'tq', 65: 'vlc', 66: 'oiz', 67: 'rqc', 68: 'sr', 69: 'zov', 70: 'vj', 71: 'u', 72: 'zud', 73: 'jcd', 74: 'zxe', 75: 'laq', 76: 'd', 77: 'c', 78: 'rw', 79: 'ssl', 80: 'ctl', 81: 'yqm', 82: 'eqr', 83: 'kw', 84: 'fg', 85: 'b', 86: 'bno', 87: 'qgi', 88: 'udu', 89: 'ow', 90: 'axz', 91: 'jpp', 92: 'k', 93: 'vr', 94: 'ax', 95: 'mlj', 96: 'egk', 97: 'keh', 98: 'qz', 99: 'om'} 

for ind in population1:
    for meaning in meanings:
        ind[meaning] = predefined_language[meaning]

for ind in population2:
    for meaning in meanings:
        ind[meaning] = predefined_language2[meaning]



In [547]:
from deap import base, creator, tools
import random
import string
import itertools

toolbox = base.Toolbox()

def create_bilingual_individual():
    individual = creator.Individual()
    individual.local_state = random.choice(meanings)
    individual.is_bilingual = 1
    # capacity for two languages
    individual.language1 = {}
    individual.language2 = {}
    individual.age = 0
    for meaning in meanings:
        word = create_word()
        individual.language1[meaning] = word
        word = create_word()
        individual.language2[meaning] = word

    return individual

toolbox.register("bilingual_individual", create_bilingual_individual)
toolbox.register("bilingual_population", tools.initRepeat, list, toolbox.bilingual_individual)


## Redefine communication methods to work with bilingual individuals

In [548]:
GLOBAL_STATE = random.choice(meanings)

"""
Adds newly heard word to individual's vocabulary 
if a randomly guessed meaning matches that of the 
speaker.
"""
def learn(individual, new_word, speaker_meaning, speaker_language):
    random_guess = random.choice(meanings)
    if random_guess == speaker_meaning:
        if individual.is_bilingual:
            if speaker_language == 1:
                individual.language1[speaker_meaning] = new_word
            else:
                individual.language2[speaker_meaning] = new_word

"""
Two individuals communicate and fitness is updates
"""
def pairwise_communication(speaker, listener, GLOBAL_STATE):
    fitness = 0
    local_meaning = speaker.local_state
    global_meaning = GLOBAL_STATE

    # choose the language used by the speaker
    if speaker.is_bilingual: # random for bilinguals
        speaker_language = 1 if random.random() < 0.5 else 2
        semantic_local = speaker.language1[local_meaning] if speaker_language == 1 else speaker.language2[local_meaning]
        semantic_global = speaker.language1[global_meaning] if speaker_language == 1 else speaker.language2[global_meaning]
    else:
        speaker_language = speaker.myLang
        semantic_local = speaker[local_meaning]
        semantic_global = speaker[global_meaning]

    decoded_local = None
    decoded_global = None

    # only interested in updating fitness for bilinguals
    if listener.is_bilingual:
        for meaning in meanings:
            if semantic_local == listener.language1.get(meaning):
                decoded_local = meaning
                if speaker_language == 1 & decoded_local == local_meaning: fitness += 1
                break
            elif semantic_local == listener.language2.get(meaning):
                decoded_local = meaning
                if speaker_language == 2 & decoded_local == local_meaning: fitness += 1
                break
            if semantic_global == listener.language1.get(meaning):
                decoded_global = meaning
                if speaker_language == 1 & decoded_global == global_meaning: fitness += 1
                break
            elif semantic_global == listener.language2.get(meaning):
                decoded_global = meaning
                if speaker_language == 2 & decoded_global == global_meaning: fitness += 1
                break

        # learning new words
        if decoded_local is None:
            learn(listener, semantic_local, local_meaning, speaker_language)
        if decoded_global is None:
            learn(listener, semantic_global, global_meaning, speaker_language)

    # penalty for same word representing different ideas
    if local_meaning != global_meaning and semantic_global == semantic_local:
        fitness -= 1

    return fitness
            
def survival_task(individual, GROUP_SIZE):
    penalty = 0
    for i in range(2 * (GROUP_SIZE - 1)): 
        meaning1, meaning2 = random.sample(meanings, 2) # two different meanings
        if individual.language1[meaning1] == individual.language1[meaning2]: # same word
            penalty += 2
        if individual.language2[meaning1] == individual.language2[meaning2]: # same word
            penalty += 2
        if individual.language1[meaning1] == individual.language2[meaning1]:
            penalty += 4
        if individual.language1[meaning2] == individual.language2[meaning2]:
            penalty += 4
    return penalty  

# evaluate group
def evaluate_group(bilingual_group, group1, group2, GROUP_SIZE):
    GLOBAL_STATE = random.choice(meanings)

    # reset fitnesses
    for ind in bilingual_group:
        ind.fitness.values = (0,)  # reset fitness

    collective = bilingual_group + group1 + group2

    # reset local_states
    for ind in collective:
        ind.local_state = random.choice(meanings)

    for speaker, listener in itertools.permutations(collective, 2):

        fitness_bonuses = []
        fitness_bonus = pairwise_communication(speaker, listener, GLOBAL_STATE)
        fitness_bonuses.append(fitness_bonus)
        if speaker.is_bilingual == 1:
            speaker.fitness.values = (speaker.fitness.values[0] + fitness_bonus,)
        if listener.is_bilingual == 1:
            listener.fitness.values = (listener.fitness.values[0] + fitness_bonus,)

    # individual survival
    for ind in bilingual_group:
        penalty = survival_task(ind, GROUP_SIZE)
        ind.fitness.values = (ind.fitness.values[0] - penalty,)

            

# initialize population

In [549]:
# create new pop
bilingual_population = toolbox.bilingual_population(n=100)

for ind in bilingual_population:
    ind.fitness.values = (0,)

toolbox.register("evaluate", evaluate_group)

GROUP_SIZE = 5
MONO_GROUP_SIZE = 50

# for i in range(500):
bilingual_groups = [bilingual_population[i:i + GROUP_SIZE] for i in range(0, len(bilingual_population), GROUP_SIZE)]
for bilingual_group in bilingual_groups:
    group1 = random.sample(population1, MONO_GROUP_SIZE)
    group2 = random.sample(population2, MONO_GROUP_SIZE)
    toolbox.evaluate(bilingual_group, group1, group2, GROUP_SIZE)

count = 0
for ind in bilingual_population:
    if ind.fitness.values[0] > 0:
        print(ind.language1, ind.fitness.values[0])
        print(ind.language2, ind.fitness.values[0])
        count += 1
print(count)

0


### params

In [550]:
from deap import algorithms
import numpy
from collections import Counter

CXPB = 0.5  # crossover probability (.5)
MUTPB = 0.01  # .01 best (100 meanings))

# mut for signals
def mutateInd(individual, indpb):
    for i in range(len(meanings)):
        if random.random() < .01:
            individual.language1[i] = create_word()
        if random.random() < .01:
            individual.language2[i] = create_word()
        if random.random() < .01: # swap signal across langs
            temp = individual.language1[i]
            individual.language1[i] = individual.language2[i]
            individual.language2[i] = temp
    return individual,

# tools
toolbox.register("mate", tools.cxUniform, indpb=0.5)
toolbox.register("mutate", mutateInd, indpb=0.1)
toolbox.register("select", tools.selRoulette) # inspired by paper
toolbox.register("tourn", tools.selTournament)
toolbox.register("randsel", tools.selRandom)
toolbox.register("stochastic_select", tools.selStochasticUniversalSampling)


stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

# for steady state
def select_next_generation(parents, offspring, max_age=3):
    combined = parents + offspring
    filtered = [ind for ind in combined if ind.age < max_age]
    selected = toolbox.tourn(filtered, len(population), 2)
    return selected

def calculate_average_agreement(population, num_meanings):
    total_agreement = 0
    # lang 1
    for i in range(num_meanings):
        gene_values = [individual.language1[i] for individual in population]
        most_common_value, count = Counter(gene_values).most_common(1)[0]
        agreement_percentage = (count / len(population)) * 100
        total_agreement += agreement_percentage
    average_agreement1 = total_agreement / num_meanings

    total_agreement = 0
    for i in range(num_meanings):
        gene_values = [individual.language2[i] for individual in population]
        most_common_value, count = Counter(gene_values).most_common(1)[0]
        agreement_percentage = (count / len(population)) * 100
        total_agreement += agreement_percentage
    average_agreement2 = total_agreement / num_meanings


    return average_agreement1, average_agreement2 

def count_duplicate_values(individual):
    value_count = Counter(individual.values())
    duplicates = [value for value, count in value_count.items() if count > 1]
    return len(duplicates)  # Return the number of unique values that are duplicates

### run

In [551]:
NUM_GENERATIONS = 1000
for gen in range(NUM_GENERATIONS):
    
    #for ES:
    MU = 100
    LAMBDA = 150

    """
    Mating selection
    """
    offspring = toolbox.randsel(bilingual_population, len(bilingual_population)) # random sel

    offspring = list(map(toolbox.clone, offspring))

    # mutation and crossover
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1.language1, child2.language1)
            toolbox.mate(child1.language2, child2.language2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    """
    Evaluate Fitness
    """
    random.shuffle(offspring)
    for ind in offspring:
        ind.fitness.values = (0,)
    bilingual_groups = [offspring[i:i + GROUP_SIZE] for i in range(0, len(offspring), GROUP_SIZE)]
    for bilingual_group in bilingual_groups:
        group1 = random.sample(population1, MONO_GROUP_SIZE)
        group2 = random.sample(population2, MONO_GROUP_SIZE)
        toolbox.evaluate(bilingual_group, group1, group2, GROUP_SIZE)

    for ind in bilingual_population:
        ind.age += 1

    """
    Survivor Selection
    """
    bilingual_population [:] = select_next_generation(bilingual_population, offspring) 

    fits = [ind.fitness.values[0] for ind in bilingual_population]

    dup_list = []

    for individual in bilingual_population:
        num_duplicates = count_duplicate_values(individual.language1) + count_duplicate_values(individual.language2)
        dup_list.append(num_duplicates)
       
    sum_dups = 0
    for dup in dup_list:
        sum_dups += dup



    # display statistics
    length = len(bilingual_population)
    print("len", length)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5

    print(f"Generation {gen}: Min {min(fits)}, Max {max(fits)}, Avg {mean}, Std {std}, Avg % of duplicate genes across participants {sum_dups / len(ind.language1) / len(bilingual_population)}")
    agreement = calculate_average_agreement(bilingual_population, 100)
    print(calculate_average_agreement(bilingual_population, 100))

    # match with source langs:
    count = 0
    for ind in bilingual_population:
        for gene in ind.language1:
            if ind.language1[gene] == predefined_language[gene]:
                count += 1
    print("Language 1 Accuracy: ", count / 10000)

    count = 0
    for ind in bilingual_population:
        for gene in ind.language2:
            if ind.language2[gene] == predefined_language2[gene]:
                count += 1
    print("Language 2 Accuracy:", count / 10000)

# print final pop
for ind in bilingual_population:
    print(ind.language1, ind.language2, ind.fitness.values)

len 100
Generation 0: Min -7.0, Max 1.0, Avg -0.03, Std 0.7273926037567333, Avg % of duplicate genes across participants 0.2032
(7.32, 7.11)
0.014
0.0144
len 100
Generation 1: Min 0.0, Max 1.0, Avg 0.01, Std 0.099498743710662, Avg % of duplicate genes across participants 0.2121
(9.28, 9.08)
0.0186
0.0188
len 100
Generation 2: Min -2.0, Max 3.0, Avg 0.11, Std 0.563826214360418, Avg % of duplicate genes across participants 0.20329999999999998
(12.07, 12.05)
0.0263
0.0225
len 100
Generation 3: Min -1.0, Max 41.0, Avg 2.19, Std 8.068079077450841, Avg % of duplicate genes across participants 0.2054
(14.91, 14.98)
0.0311
0.0312
len 100
Generation 4: Min -4.0, Max 22.0, Avg 0.25, Std 2.2555487137279924, Avg % of duplicate genes across participants 0.1956
(18.88, 19.07)
0.0375
0.0352
len 100
Generation 5: Min -4.0, Max 2.0, Avg 0.31, Std 0.7835177087979569, Avg % of duplicate genes across participants 0.19829999999999998
(21.52, 22.35)
0.0422
0.036
len 100
Generation 6: Min -2.0, Max 3.0, Avg 